<a href="https://colab.research.google.com/github/pouuh/prospect_bayes/blob/master/ExportImages8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#改变了range(0-9000),去了云，有modis mask
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
from pprint import pprint

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !earthengine authenticate 

In [ ]:
ee.Authenticate()
ee.Initialize()

# # Test the earthengine command by getting help on upload.
# !earthengine upload image -h

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=7-CoeNs1vkB1_mTDemfxU-AUy59WFLuqEWDnT0-vof8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7eFRxvx3DxLnqUVpZ43sspK3QzCEPLqy4sd3RmnKZBfX9LjWO5wRQ

Successfully saved authorization token.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def getHistogram2(image,region,flag):
  if flag==1:#LST
    redux = image.reduceRegion(**{
      #'reducer': ee.Reducer.fixedHistogram(7500,65535,32),
      'reducer': ee.Reducer.fixedHistogram(12400,15600,32),
      'geometry': region.geometry(),
      'scale': 500})
  else:
    redux = image.reduceRegion(**{
    #'reducer': ee.Reducer.fixedHistogram(-100,16000,32),
    'reducer': ee.Reducer.fixedHistogram(0,9000,32),
    'geometry': region.geometry(),
    'scale': 500})

  return redux;

In [ ]:
#transform FeatureCollection
def feature_func_1(element):
   return ee.Feature(None,{'prop':element})
#featureCollection = ee.FeatureCollection(test.map(feature_func_1))

In [ ]:
def export_hists(region,imgs,name,flag):
  #flag: 0 reperents R; 1 reperents LST
  imgList=imgs.toList(1e4)
  size_img=imgList.size()

  def func_1(i):
    i=ee.Number(i);
    image=ee.Image(imgList.get(i));
    #然后对这个image操作，再返回结果
    return getHistogram2(image,region,flag);

  test=ee.List.sequence(0,size_img.subtract(1)).map(func_1)
  featureCollection = ee.FeatureCollection(test.map(feature_func_1))

  if flag==0:
    task = ee.batch.Export.table.toDrive(**{
      'collection': featureCollection,
      'description': 'Col'+name,
      'folder': 'ImgColHist_masked_modis_cloud_newrange',
      'fileFormat': 'GeoJSON'
    })
  else:
    task = ee.batch.Export.table.toDrive(**{
      'collection': featureCollection,
      'description': 'Lst'+name,
      'folder': 'ImgLstHist_masked_modis_cloud_newrange',
      'fileFormat': 'GeoJSON'
    })
  task.start()
  time.sleep(3)
  # while task.status()['state'] == 'RUNNING':
  #   print('Running...')
  #   # Perhaps task.cancel() at some point.
  #   time.sleep(10)
  # print ('Done.', task.status())

In [ ]:
locations = pd.read_csv('/content/drive/MyDrive/earthengine/locations_final.csv',header=None)
print(locations)
#我这里需要将原始的产量跟提取的反射率匹配上，因为数据是所有年份一起提取的，但是参量确实每一年的

       0    1        2       3
0      1   47  -87.114  32.334
1      1   65  -87.623  32.753
2      1   85  -86.651  32.148
3      1  105  -87.294  32.639
4      1  119  -88.200  32.597
...   ..  ...      ...     ...
1829  48  189 -101.820  34.068
1830  48   17 -102.830  34.068
1831  48   79 -102.830  33.608
1832  48  107 -101.300  33.609
1833  48  303 -101.820  33.611

[1834 rows x 4 columns]


In [ ]:
#cloud mask function
def bitwiseExtract(value, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return value.rightShift(fromBit).bitwiseAnd(mask)


def cloudfree_MOD11A2(image):
  qa1 = image.select('QC_Day')
  cloudState = bitwiseExtract(qa1, 2, 3) 
  #cloudShadowState = bitwiseExtract(qa, 2, 2)
  #cirrusState = bitwiseExtract(qa, 8, 9)
  qa2 = image.select('QC_Night')
  cloudState2 = bitwiseExtract(qa2, 2, 3) 
  mask = cloudState.eq(0) and cloudState2.eq(0)
    
  return image.updateMask(mask)  

def cloudfree_mod09a1(image):
  qa = image.select('StateQA')
  cloudState = bitwiseExtract(qa, 0, 1) 
  cloudShadowState = bitwiseExtract(qa, 2, 2)
  cirrusState = bitwiseExtract(qa, 8, 9)
  mask = cloudState.eq(0) and (cloudShadowState.eq(0)) and (cirrusState.eq(0))
    
  return image.updateMask(mask)  

In [ ]:
#soybean mask
def soybeans_cdl(img):
  date_t=img.date()
  data_range2=date_t.getRange('year')
  CDL_imgs = ee.ImageCollection("USDA/NASS/CDL")\
    .filterDate(data_range2.start(),data_range2.end()).select([0])
    
  CDL_img=ee.Image(CDL_imgs.mosaic())
  return img.updateMask(CDL_img.eq(5))

def soybeans_modis(img):
  date_t=img.date()
  data_range2=date_t.getRange('year')
  CDL_imgs = ee.ImageCollection('MODIS/006/MCD12Q1')\
    .filterDate(data_range2.start(),data_range2.end()).select([0])
    
  CDL_img=ee.Image(CDL_imgs.mosaic())
  return img.updateMask(CDL_img.eq(12))

# imgcol_masked=imgcol.map(soybeans_modis)
# imgcol2_masked=imgcol2.map(soybeans_cdl)

imgcol = ee.ImageCollection('MODIS/006/MOD09A1')\
  .filterDate('2003-01-01','2015-12-31')#.select([0,1,2,3,4,5,6])

imglst = ee.ImageCollection("MODIS/006/MOD11A2")\
    .filterDate('2003-01-01','2015-12-31')\
    .select(['LST_Day_1km','LST_Night_1km'])

#cloud remove
imgcol = imgcol.map(cloudfree_mod09a1)\
  .select([0,1,2,3,4,5,6]);

imgcol_masked=imgcol.map(soybeans_modis)
imglst_masked=imglst.map(soybeans_modis)

pprint(imgcol_masked.serialize())

# imglst=ee.ImageCollection("MODIS/006/MOD11A2")\
#       .filterDate('2002-12-31','2016-06-30')\
#       .select(['LST_Day_1km','LST_Night_1km']);      
#print(imgcoll.serialize())

('{"result": "0", "values": {"2": {"functionInvocationValue": {"functionName": '
 '"Image.select", "arguments": {"bandSelectors": {"constantValue": [0]}, '
 '"input": {"argumentReference": "_MAPPING_VAR_0_0"}}}}, "3": '
 '{"functionInvocationValue": {"functionName": "DateRange", "arguments": '
 '{"start": {"functionInvocationValue": {"functionName": "Date.getRange", '
 '"arguments": {"date": {"functionInvocationValue": {"functionName": '
 '"Image.date", "arguments": {"image": {"argumentReference": '
 '"_MAPPING_VAR_1_0"}}}}, "unit": {"constantValue": "year"}}}}}}}, "4": '
 '{"constantValue": "system:time_start"}, "1": {"functionInvocationValue": '
 '{"functionName": "Image.updateMask", "arguments": {"image": '
 '{"argumentReference": "_MAPPING_VAR_1_0"}, "mask": '
 '{"functionInvocationValue": {"functionName": "Image.eq", "arguments": '
 '{"image1": {"functionInvocationValue": {"functionName": '
 '"ImageCollection.mosaic", "arguments": {"collection": '
 '{"functionInvocationValue": {"f

In [ ]:
#pprint(imgcol_masked.first().getInfo()['bands'][0]['id'])

# import folium
# print(folium.__version__)
# # Use folium to visualize the imagery.
# mapid = imgcols_masked.median().getMapId({'bands': ['LST_Day_1km']})
# map = folium.Map()#location=[38., -122.5])

# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='median composite',
#   ).add_to(map)
# map.add_child(folium.LayerControl())
# map

In [ ]:
idxs_state=locations[0]
idxs_county=locations[1]
#print(locations.dtypes)
dataset = ee.FeatureCollection('TIGER/2016/Counties')
for item in locations.index:
#for item in range(828,1834):
  idx1=idxs_state.get(item)
  idx2=idxs_county.get(item)

  state_str=str(idx1)
  if idx1<10:
    state_str='0'+state_str
  #print(state_str)

  county_str=str(idx2)
  if idx2<100:
    county_str='0'+county_str
    if idx2<10:
      county_str='0'+county_str
  #print(county_str)

  name=str(item)+'_'+state_str+'_'+county_str
  #print(name)
  
  region = dataset.filterMetadata('STATEFP', 'equals', state_str)
  region = ee.FeatureCollection(region).filterMetadata('COUNTYFP', 'equals', county_str)
  region = ee.Feature(region.first())

  #x=region.getInfo()
  #print(x['properties'])
  
  while True:
    try:
        export_hists(region,imgcol_masked,name,0)
    except:
        print('retry')
        time.sleep(10)
        continue
    #################################
    try:
        export_hists(region,imglst_masked,name,1)
    except:
        print('retry')
        time.sleep(10)
        continue
    break
  print(name)

828_28_015
829_28_051
830_28_089
831_28_097
832_28_121
833_28_123
834_28_155
835_28_017
836_28_025
837_28_087
838_28_095
839_28_103
840_28_053
841_28_055
842_28_083
843_28_125
844_28_133
845_28_151
846_28_163
847_28_009
848_28_013
849_28_033
850_28_043
851_28_071
852_28_093
853_28_107
854_28_137
855_28_161
856_28_003
857_28_057
858_28_081
859_28_115
860_28_117
861_28_139
862_28_141
863_28_145
864_28_129
865_28_147
866_28_101
867_28_049
868_28_063
869_28_149
870_28_011
871_28_027
872_28_119
873_28_135
874_28_143
875_29_019
876_29_051
877_29_059
878_29_105
879_29_125
880_29_131
881_29_141
882_29_151
883_29_159
884_29_167
885_29_195
886_29_055
887_29_071
888_29_073
889_29_099
890_29_157
891_29_183
892_29_187
893_29_186
894_29_033
895_29_041
896_29_117
897_29_171
898_29_211
899_29_007
900_29_111
901_29_173
902_29_021
903_29_025
904_29_049
905_29_061
906_29_147
907_29_165
908_29_227
909_29_223
910_29_225
911_29_031
912_29_069
913_29_143
914_29_155
915_29_201
916_29_207
917_29_009
918_29_011

In [ ]:
#pprint(ee.batch.Task.list()[0:1500])

In [ ]:
#!earthengine task cancel all 